In [1]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import os
from dotenv import load_dotenv
from find_image import FindImage
import botocore
import json

2023-10-12 14:21:08.710151: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-12 14:21:08.710191: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-12 14:21:08.710224: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-12 14:21:08.719149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 14:21:09.713217: W tensorflow/compiler/

In [2]:
tf.config.list_physical_devices('GPU')

2023-10-12 14:21:11.827298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 14:21:11.848650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 14:21:11.848844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.debugging.set_log_device_placement(True)

In [4]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [5]:
load_dotenv()

True

In [ ]:
# Loading model
loaded_model = tf.keras.models.load_model(os.getenv('MODEL_PATH'))

In [7]:
with open('class_names.json', 'r') as file:
    class_names_generated = json.load(file)

In [10]:
def match_pet():
  # Carregando e redimensionando a imagem
  img = image.load_img(os.getenv('PATH_DOWNLOADED_IMG'), target_size=(224, 224))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0  # Normalização

  # Fazendo a previsão
  prediction = loaded_model.predict(img_array)
  
  # Obtendo a classe prevista
  predicted_class_index = np.argmax(prediction)

  # Obtendo o nome da classe
  predicted_class = class_names_generated[f"{predicted_class_index}"]


  # Imprimindo a classe prevista
  print(predicted_class)

In [11]:
s3 = FindImage(flag=True, bucket_name=os.getenv("BUCKET_S3"), resource_name="s3")

try:
    print(os.getenv('PATH_DOWNLOADED_IMG'))
    #s3.download(os.getenv('FILE_KEY'), os.getenv('PATH_DOWNLOADED_IMG'))
    print(f"Arquivo baixado com sucesso.")
    match_pet()
except botocore.exceptions.NoCredentialsError:
    print("Credenciais não encontradas ou inválidas.")
except botocore.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == "404":
        print("O arquivo não foi encontrado no bucket.")
    else:
        print("Erro desconhecido ao baixar o arquivo:", e)

/home/patrick/SPTECH/TCC/grupo11-4cco-tcc/src/inteligencia_artificial/imgs_test/dog1.jpg
Arquivo baixado com sucesso.
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
start: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
stop: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
step: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
RangeDataset: (RangeDataset): /job:localhost/replica:0/task:0/device:CPU:0
handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
input__dataset: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
count: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
RepeatDataset: (RepeatDataset): /job:localhost/replica:0/task:0/device:CPU:0
handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
input__dataset: (_Arg): /job:localhost

2023-10-12 14:22:02.590892: I tensorflow/core/common_runtime/placer.cc:125] start: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:02.590921: I tensorflow/core/common_runtime/placer.cc:125] stop: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:02.590927: I tensorflow/core/common_runtime/placer.cc:125] step: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:02.590937: I tensorflow/core/common_runtime/placer.cc:125] RangeDataset: (RangeDataset): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:02.590944: I tensorflow/core/common_runtime/placer.cc:125] handle_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:02.592956: I tensorflow/core/common_runtime/placer.cc:125] input__dataset: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:02.592972: I tensorflow/core/common_runtime/placer.cc:125] count: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:2

iterator: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
sequential_mobilenetv2_1_00_224_conv1_conv2d_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilenetv2_1_00_224_bn_conv1_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilenetv2_1_00_224_bn_conv1_readvariableop_1_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilenetv2_1_00_224_bn_conv1_fusedbatchnormv3_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilenetv2_1_00_224_bn_conv1_fusedbatchnormv3_readvariableop_1_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilenetv2_1_00_224_expanded_conv_depthwise_depthwise_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilenetv2_1_00_224_expanded_conv_depthwise_bn_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
sequential_mobilene

2023-10-12 14:22:03.332985: I tensorflow/core/common_runtime/placer.cc:125] iterator: (_Arg): /job:localhost/replica:0/task:0/device:CPU:0
2023-10-12 14:22:03.333006: I tensorflow/core/common_runtime/placer.cc:125] sequential_mobilenetv2_1_00_224_conv1_conv2d_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-10-12 14:22:03.333038: I tensorflow/core/common_runtime/placer.cc:125] sequential_mobilenetv2_1_00_224_bn_conv1_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-10-12 14:22:03.333042: I tensorflow/core/common_runtime/placer.cc:125] sequential_mobilenetv2_1_00_224_bn_conv1_readvariableop_1_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-10-12 14:22:03.333045: I tensorflow/core/common_runtime/placer.cc:125] sequential_mobilenetv2_1_00_224_bn_conv1_fusedbatchnormv3_readvariableop_resource: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2023-10-12 14:22:03.333049: I tensorflow/core/common_runti

Executing op __inference_predict_function_21697 in device /job:localhost/replica:0/task:0/device:GPU:0


2023-10-12 14:22:03.815039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904


1/1 [==============================] - 2s 2s/step
dog1
